In [14]:
import os, random, json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler

import optuna
from metrics import metrics_from_preds
from preprocess import *

# Carga de datos

In [16]:
def set_seed(seed=42):
    SEED = seed
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    os.environ["PYTHONHASHSEED"] = str(seed)

In [17]:
set_seed(42)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cpu


In [19]:
vids_training, labels_training, vids_val, labels_val, vids_test, labels_test = load_data()

D:\A_Investigacion\post_semillero\ai_paper_js_gl\data


In [20]:
class VideoDataset(Dataset):
    def __init__(self, videos, labels):
        self.videos = videos
        self.labels = labels

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        x = torch.tensor(np.array(self.videos[idx]), dtype=torch.float32)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

In [21]:
set_seed()

train_dataset = VideoDataset(vids_training, labels_training)
val_dataset   = VideoDataset(vids_val, labels_val)

g = torch.Generator().manual_seed(42)
perm = torch.randperm(len(train_dataset), generator=g).tolist()
train_sampler = SubsetRandomSampler(perm)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=0, sampler=train_sampler)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

# Modelo Transformer

In [22]:
class SelfAttentionBlock(nn.Module):
    def __init__(self, embed_dim=258, num_heads=4, dropout=0.1):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

    def forward(self, x):
        attn_out, _ = self.attn(x, x, x)
        x = self.norm1(x + attn_out)
        ff_out = self.ff(x)
        x = self.norm2(x + ff_out)
        return x

In [23]:
class TransformerModel(nn.Module):
    def __init__(self, in_features=258, num_classes=4, num_heads=4, layers=2, hidden=128, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(in_features, hidden)
        self.layers = nn.ModuleList([
            SelfAttentionBlock(embed_dim=hidden, num_heads=num_heads, dropout=dropout) for _ in range(layers)
        ])
        self.head = nn.Linear(hidden, num_classes)

    def forward(self, x):
        x = self.input_proj(x)
        for layer in self.layers:
            x = layer(x)
        x = x.mean(dim=1)
        return self.head(x)

# Optimización de Hiperparametros

In [27]:
def optimize_transformer(train_loader, val_loader, n_trials=50, max_epochs=80, save_prefix="hpo_transformer"):
    set_seed()
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="maximize", sampler=sampler, pruner=optuna.pruners.MedianPruner())
    all_trials = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def objective(trial):
        hidden  = trial.suggest_categorical("hidden",[96,128,160,192])
        layers  = trial.suggest_int("layers", 2, 5)
        num_heads = trial.suggest_categorical("num_heads", [2, 4, 8])
        drop    = trial.suggest_float("dropout", 0.0, 0.5)
        lr      = trial.suggest_float("lr", 5e-5, 5e-3, log=True)
        wd      = trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)

        model = TransformerModel(hidden=hidden, layers=layers, num_heads=num_heads, dropout=drop).to(device)
        crit = nn.CrossEntropyLoss()
        opt  = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

        best_val = 0.0
        for epoch in range(max_epochs):
            model.train()
            for X, y in train_loader:
                X, y = X.to(device), y.to(device)
                opt.zero_grad()
                out = model(X)
                loss = crit(out, y)
                loss.backward(); opt.step()
            model.eval()
            y_true, y_pred = [], []
            with torch.no_grad():
                for X, y in val_loader:
                    X, y = X.to(device), y.to(device)
                    pred = model(X).argmax(1)
                    y_true.extend(y.cpu().numpy()); y_pred.extend(pred.cpu().numpy())
            val_acc = metrics_from_preds(y_true, y_pred)["accuracy"]
            if val_acc > best_val: best_val = val_acc
            trial.report(val_acc, epoch)
            if trial.should_prune(): raise optuna.TrialPruned()

        all_trials.append({"trial": trial.number, "params": trial.params, "best_val_acc": best_val})
        return best_val

    study.optimize(objective, n_trials=n_trials)
    with open(f"{save_prefix}_all.json","w") as f: json.dump(all_trials,f,indent=2)
    with open(f"{save_prefix}_best.json", "w") as f: json.dump({"best_params":study.best_params, "best_value":study.best_value}, f, indent=2)
    return study

In [28]:
study_transformer = optimize_transformer(train_loader, val_loader, n_trials=40, max_epochs=100)

[I 2025-11-30 19:41:07,002] A new study created in memory with name: no-name-396d6f8d-7cd9-43cc-9f1e-04a3b3e05389
[I 2025-11-30 19:42:32,832] Trial 0 finished with value: 0.9333333333333333 and parameters: {'hidden': 128, 'layers': 2, 'num_heads': 8, 'dropout': 0.3005575058716044, 'lr': 0.0013035123791853842, 'weight_decay': 1.3934502251337584e-10}. Best is trial 0 with value: 0.9333333333333333.
[I 2025-11-30 19:43:36,750] Trial 1 finished with value: 0.9277777777777778 and parameters: {'hidden': 96, 'layers': 2, 'num_heads': 4, 'dropout': 0.14561457009902096, 'lr': 0.0008369042894376068, 'weight_decay': 9.472334467618544e-10}. Best is trial 0 with value: 0.9333333333333333.
[I 2025-11-30 19:45:21,123] Trial 2 finished with value: 0.9277777777777778 and parameters: {'hidden': 192, 'layers': 2, 'num_heads': 4, 'dropout': 0.3037724259507192, 'lr': 0.0001096524277832185, 'weight_decay': 2.853390105240223e-10}. Best is trial 0 with value: 0.9333333333333333.
[I 2025-11-30 19:47:25,322] Tr

In [ ]:
print("Mejor resultado:", study_transformer.best_value)
print("Mejores hiperparámetros:", study_transformer.best_params)

Mejor resultado: 0.95
Mejores hiperparámetros: {'hidden': 96, 'layers': 5, 'num_heads': 4, 'dropout': 0.26136641469099703, 'lr': 0.00035813934999486707, 'weight_decay': 1.5063777323554432e-10}


In [30]:
import json
from metrics import metrics_from_preds # Corregido de "metrics_from_"
import torch
import torch.nn as nn
import torch.optim as optim

# --- Cargar los mejores hiperparámetros ---
print("Cargando los mejores hiperparámetros desde hpo_transformer_best.json...")
try:
    with open("hpo_transformer_best.json", "r") as f:
        best_hyperparams = json.load(f)["best_params"]
    print("Hiperparámetros cargados:")
    print(best_hyperparams)
except FileNotFoundError:
    print("Error: El archivo 'hpo_transformer_best.json' no fue encontrado.")
    print("Por favor, asegúrate de que el archivo exista y contenga los mejores hiperparámetros.")
    # Detener la ejecución si el archivo no existe.
    # En un notebook, puedes simplemente no ejecutar las celdas siguientes.
    raise

# --- Configurar y entrenar el mejor modelo ---
set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instanciar el modelo con los mejores hiperparámetros
best_model = TransformerModel(
    hidden=best_hyperparams["hidden"],
    layers=best_hyperparams["layers"],
    num_heads=best_hyperparams["num_heads"],
    dropout=best_hyperparams["dropout"]
).to(device)

# Definir criterio de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(
    best_model.parameters(),
    lr=best_hyperparams["lr"],
    weight_decay=best_hyperparams["weight_decay"]
)

# Entrenamiento del modelo
epochs = 100  # Usamos 100 épocas como en la búsqueda de hiperparámetros
print(f"\nEntrenando el mejor modelo durante {epochs} épocas...")

for epoch in range(epochs):
    best_model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = best_model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

print("Entrenamiento completado.")

# --- Evaluación del modelo y cálculo de métricas ---
best_model.eval()
all_metrics = {}

# Función para evaluar un dataloader
def evaluate_model(loader, model_name):
    y_true, y_pred = [], []
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            predictions = best_model(X).argmax(1)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())
    
    metrics = metrics_from_preds(y_true, y_pred)
    all_metrics[model_name] = metrics

# Evaluar en conjunto de entrenamiento y validación
print("\n--- Métricas del Modelo Final ---")
evaluate_model(train_loader, "Train")
evaluate_model(val_loader, "Validation")

# Imprimir métricas de forma legible
for split_name, metrics in all_metrics.items():
    print(f"\nResultados en el conjunto de {split_name}:")
    for metric_name, value in metrics.items():
        if isinstance(value, float):
            print(f"  {metric_name}: {value:.4f}")
        else:
            print(f"  {metric_name}: {value}")

# --- Guardar el modelo entrenado ---
model_save_path = "transformer_best_model.pth"
torch.save(best_model.state_dict(), model_save_path)
print(f"\nModelo guardado exitosamente en: {model_save_path}")

Cargando los mejores hiperparámetros desde hpo_transformer_best.json...
Hiperparámetros cargados:
{'hidden': 96, 'layers': 5, 'num_heads': 4, 'dropout': 0.26136641469099703, 'lr': 0.00035813934999486707, 'weight_decay': 1.5063777323554432e-10}

Entrenando el mejor modelo durante 100 épocas...
Epoch [10/100], Loss: 0.4147
Epoch [20/100], Loss: 0.1339
Epoch [30/100], Loss: 0.0675
Epoch [40/100], Loss: 0.3449
Epoch [50/100], Loss: 0.0182
Epoch [60/100], Loss: 0.0031
Epoch [70/100], Loss: 0.1464
Epoch [80/100], Loss: 0.0022
Epoch [90/100], Loss: 0.0010
Epoch [100/100], Loss: 0.0007
Entrenamiento completado.

--- Métricas del Modelo Final ---

Resultados en el conjunto de Train:
  accuracy: 1.0000
  precision_macro: 1.0000
  recall_macro: 1.0000
  specificity_macro: 1.0000
  f1_macro: 1.0000
  balanced_accuracy: 1.0000
  confusion_matrix: [[255, 0, 0, 0], [0, 255, 0, 0], [0, 0, 255, 0], [0, 0, 0, 255]]

Resultados en el conjunto de Validation:
  accuracy: 0.9333
  precision_macro: 0.9373
  